In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import RandomOverSampler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🟢 Using device: {device}")

🟢 Using device: cuda


In [2]:
cols = [
    "loan_id", "dependents", "education", "self_employed", "imcome_annum",
    "loan_amount", "loan_term", "cibil_score", "residential_assets_value",
    "commercial_assets_value", "luxury_assets_value", "bank_asset_value", "status"
]

df = pd.read_csv("loan_approval_dataset_processed.csv", names=cols, header=0)

In [3]:
if "loan_id" in df.columns:
    df = df.drop("loan_id", axis=1)

In [4]:
label_enc = LabelEncoder()
df["education"] = label_enc.fit_transform(df["education"])
df["self_employed"] = label_enc.fit_transform(df["self_employed"])

In [6]:
X = df.drop("status", axis=1)
y = df["status"]


In [7]:
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X, y)


In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_res)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_res, test_size=0.2, random_state=42
)

In [10]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

In [11]:
class LoanNN(nn.Module):
    def __init__(self, input_dim):
        super(LoanNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

In [12]:
model = LoanNN(X_train.shape[1]).to(device)

In [13]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [15]:
epochs = 50
for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(train_loader):.4f}")

Epoch [1/50], Loss: 0.5709
Epoch [2/50], Loss: 0.3060
Epoch [3/50], Loss: 0.2366
Epoch [4/50], Loss: 0.2171
Epoch [5/50], Loss: 0.2062
Epoch [6/50], Loss: 0.1905
Epoch [7/50], Loss: 0.1805
Epoch [8/50], Loss: 0.1739
Epoch [9/50], Loss: 0.1665
Epoch [10/50], Loss: 0.1655
Epoch [11/50], Loss: 0.1569
Epoch [12/50], Loss: 0.1531
Epoch [13/50], Loss: 0.1425
Epoch [14/50], Loss: 0.1436
Epoch [15/50], Loss: 0.1391
Epoch [16/50], Loss: 0.1371
Epoch [17/50], Loss: 0.1324
Epoch [18/50], Loss: 0.1512
Epoch [19/50], Loss: 0.1199
Epoch [20/50], Loss: 0.1242
Epoch [21/50], Loss: 0.1220
Epoch [22/50], Loss: 0.1399
Epoch [23/50], Loss: 0.1141
Epoch [24/50], Loss: 0.1145
Epoch [25/50], Loss: 0.1061
Epoch [26/50], Loss: 0.1063
Epoch [27/50], Loss: 0.1037
Epoch [28/50], Loss: 0.1050
Epoch [29/50], Loss: 0.1052
Epoch [30/50], Loss: 0.0979
Epoch [31/50], Loss: 0.0952
Epoch [32/50], Loss: 0.0935
Epoch [33/50], Loss: 0.0937
Epoch [34/50], Loss: 0.0891
Epoch [35/50], Loss: 0.0879
Epoch [36/50], Loss: 0.0911
E

In [17]:
from sklearn.metrics import classification_report

# Set model to evaluation mode
model.eval()

# Disable gradient calculation for evaluation
with torch.no_grad():
    # Get predictions on the test set
    y_pred_probs = model(X_test_tensor)
    y_pred = (y_pred_probs >= 0.5).float()  # Convert probabilities to 0 or 1

# Move tensors to CPU before converting to numpy
y_true_np = y_test_tensor.cpu().numpy()
y_pred_np = y_pred.cpu().numpy()

# Print classification report
print(classification_report(y_true_np, y_pred_np, target_names=["Rejected", "Approved"]))


              precision    recall  f1-score   support

    Rejected       0.98      0.96      0.97       561
    Approved       0.96      0.98      0.97       502

    accuracy                           0.97      1063
   macro avg       0.97      0.97      0.97      1063
weighted avg       0.97      0.97      0.97      1063



In [18]:
torch.save({
    "model_state_dict": model.state_dict(),
    "scaler": scaler,
    "label_enc": label_enc
}, "loan_nn_model_gpu.pth")

print("\n✅ Neural network trained on GPU and saved as 'loan_nn_model_gpu.pth'.")


✅ Neural network trained on GPU and saved as 'loan_nn_model_gpu.pth'.
